In [2]:
import pandas as pd
import numpy as np
import os
import socket
from tqdm import tqdm

# libraries to parallelize code
import multiprocessing as mp
from joblib import Parallel, delayed

In [4]:
path_training= "./data/training_data.csv"

df = pd.read_csv(path_training, error_bad_lines=False,encoding='utf-8',sep="\t")
df

,eval,set_ratio,sort_ratio,ratio,overlap,soundex,score,max,reverse,x_name,y_name,x_id,y_id
0,1,93,93,93,1.0,1,281.0,281.0,0,various artists,various aritsts,1,4883
1,1,100,100,100,1.0,1,302.0,302.0,0,various artists,various artists,1,4884
2,1,100,100,100,1.0,1,302.0,302.0,0,massive attack,massive attack,4,2982
3,1,100,100,100,1.0,1,302.0,302.0,0,robert miles,robert miles,9,3922
4,1,100,100,100,1.0,1,302.0,302.0,0,bob dylan,bob dylan,17,528
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7649,1,100,100,100,1.0,1,302.0,302.0,0,joe cocker & jennifer warnes,joe cocker & jennifer warnes,1388122,2294
7650,1,100,100,100,1.0,1,302.0,302.0,0,nirvana,nirvana,1388316,3360
7651,1,100,100,100,1.0,1,302.0,302.0,0,vip,vip,1388405,4855
7652,1,100,100,47,1.0,0,248.0,248.0,1,suzanne collins,collins suzanne,1388451,875


In [5]:
x = df[["x_id","x_name"]]
y = df[['y_id','y_name']]

In [15]:
x_final = x.drop_duplicates()
y_final = y.drop_duplicates()

In [11]:
y.duplicated(["y_id"]).sum()

2506

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
#ngram best result

import re
import time
from ftfy import fix_text #  text c<leaning for decode issues..
from scipy.sparse import csr_matrix # may not be required 
from scipy.sparse import rand # may not be required
import nmslib

In [54]:
def ngrams(string, n=2):
    """Takes an input string, cleans it and converts to ngrams. 
    This script is focussed on cleaning UK company names but can be made generic by removing lines below"""
    string = str(string)
    string = string.lower() # lower case
    string = fix_text(string) # fix text
    string = string.split('t/a')[0] # split on 'trading as' and return first name only
    #string = string.split('trading as')[0] # split on 'trading as' and return first name only
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    chars_to_remove = [")","(",".","|","[","]","{","}","'","-"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']' #remove punc, brackets etc...
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    #string = string.replace('limited', 'ltd')
    #string = string.replace('public limited company', 'plc')
    #string = string.replace('united kingdom', 'uk')
    #string = string.replace('community interest company', 'cic')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [55]:
article_name = list(y_final['y_name'][:].unique())
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(article_name)

# messy data => EX: popsike data
messy_tracks = list(x_final['x_name'][:].dropna().unique())
messy_tf_idf_matrix = vectorizer.transform(messy_tracks)

In [56]:
# create a random matrix to index
data_matrix = tf_idf_matrix#[0:1000000]

# Set index parameters
# These are the most important ones
M = 80
efC = 1000

num_threads = 4 # adjust for the number of threads
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='simple_invindx', space='negdotprod_sparse_fast', data_type=nmslib.DataType.SPARSE_VECTOR) 

index.addDataPointBatch(data_matrix)
# Create an index
start = time.time()
index.createIndex() 
end = time.time() 
print('Indexing time = %f' % (end-start))

# c'est là que ça prend le plus de temps
# Number of neighbors => use Knn
num_threads = 4
K=1
query_matrix = messy_tf_idf_matrix
start = time.time() 
query_qty = query_matrix.shape[0]
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty))

mts =[]
for i in range(len(nbrs)):
  origional_nm = messy_tracks[i] #changer les noms mais du coup le original c'est bien musicbrainz qui est le original et popsike le atched name
  try:
    matched_nm   = article_name[nbrs[i][0][0]]
    conf         = nbrs[i][1][0]
  except:
    matched_nm   = "no match found"
    conf         = None
  mts.append([origional_nm,matched_nm,conf])

mts = pd.DataFrame(mts,columns=['origional_name','matched_name','conf'])
mts['conf'] = -mts['conf'] #otherwise -1 represent confidence = 100%
#results = df_CF.merge(mts,left_on='buyer',right_on='origional_name')
print(mts)
mts = mts.sort_values(["conf"], ascending=False)

Indexing time = 0.008762
kNN time total=0.512091 (sec), per query=0.000124 (sec), per query adjusted for thread number=0.000497 (sec)
                    origional_name                  matched_name      conf
0                  various artists               various artists  1.000000
1                   massive attack                massive attack  1.000000
2                     robert miles                  robert miles  1.000000
3                        bob dylan                     bob dylan  1.000000
4                        tom waits                     waits tom  1.000000
...                            ...                           ...       ...
4116                  bradley hale               bradley charles  0.716195
4117                     t pinnock                  pinnock & ec  0.622408
4118                   dan padilla                  juan padilla  0.729556
4119                   rino christ                      christer  0.620590
4120  joe cocker & jennifer warnes  joe c

In [57]:
df_test = mts.merge(x_final, how="left",left_on="origional_name", right_on="x_name")
df_test = df_test.merge(y_final, how="left",left_on="matched_name", right_on="y_name")
del df_test['x_name']
del df_test['y_name']
df_test

,origional_name,matched_name,conf,x_id,y_id
0,gavin degraw,degraw gavin,1.000000,106120,1077
1,wilkinson,wilkinson,1.000000,722265,5040
2,chris avedon & coon,chris avedon & coon,1.000000,827178,789
3,captain hollywood project,captain hollywood project,1.000000,1879,711
4,ariana grande,grande ariana,1.000000,823336,1845
...,...,...,...,...,...
6285,rka markov,parov stelar,0.412739,1068440,3553
6286,mel larned,bill medley & jennifer warnes,0.409694,716277,453
6287,tj upshaw,tjr,0.357160,1233772,4725
6288,tnu kaljuste,estonian pcc & estonian national so & kaljuste,0.353035,242257,1476


In [58]:
df_final = df_test.merge(df, how="left",left_on="x_id", right_on="x_id")
df_final = df_final.drop(columns=['eval','set_ratio','sort_ratio','ratio','overlap','soundex','score','max','reverse'])
df_final

,origional_name,matched_name,conf,x_id,y_id_x,x_name,y_name,y_id_y
0,gavin degraw,degraw gavin,1.000000,106120,1077,gavin degraw,degraw gavin,1077
1,wilkinson,wilkinson,1.000000,722265,5040,wilkinson,wilkinson,5040
2,chris avedon & coon,chris avedon & coon,1.000000,827178,789,chris avedon & coon,chris avedon & coon,789
3,captain hollywood project,captain hollywood project,1.000000,1879,711,captain hollywood project,captain hollywood project,711
4,ariana grande,grande ariana,1.000000,823336,1845,ariana grande,ariana grande,225
...,...,...,...,...,...,...,...,...
8017,rka markov,parov stelar,0.412739,1068440,3553,rka markov,gergiev & pape & markov & semishkur &,1772
8018,mel larned,bill medley & jennifer warnes,0.409694,716277,453,mel larned,mdel bjarne,3243
8019,tj upshaw,tjr,0.357160,1233772,4725,tj upshaw,latry & upshaw & muraro & boulez & nagano & &,2678
8020,tnu kaljuste,estonian pcc & estonian national so & kaljuste,0.353035,242257,1476,tnu kaljuste,prt & estonian pcc & estonian nsc & kaljuste,3773


In [59]:
match_true = 0
for i in range(len(df_final[:])):
    if df_final['y_id_x'].loc[i] == df_final['y_id_y'].loc[i]:
        match_true += 1

accuracy_score = round(match_true/len(df_final[:]),4)
print(f'Accuracy score: {accuracy_score*100}%') #accuracy score: 77%

Accuracy score: 70.94%
